# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [30]:
from xml.etree import ElementTree as ET
import numpy as np
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [361]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [362]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [363]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [364]:
document = ET.parse( './data/mondial_database.xml' )

In [365]:
root = document.getroot()
root.tag

'mondial'

In [366]:
def find_cities(e):
    for elem in e.findall('city'):
        yield elem
    for p in e.findall('province'):
        for elem in p.findall('city'):
            yield elem

country_dict = {}
city_dict = {}

ethnic_list = []
pop_list = []

for country in root.iter('country'):
    country_name = country.find('name').text
    country_dict[country_name] = {}
    country_dict[country_name]['name'] = country_name
    country_dict[country_name]['car_code'] = country.attrib['car_code']
    try:
        country_dict[country_name]['infant_mortality'] = country.find('infant_mortality').text
    except:
        country_dict[country_name]['infant_mortality'] = np.nan
    
    country_dict[country_name]['area'] = country.attrib.get('area', np.nan)
    country_dict[country_name]['capital'] = country.attrib.get('capital', np.nan)
    
    # Populate country information in enthnic_list
    for elem in country.findall('ethnicgroup'):
        ethn = {}
        ethn_name = elem.text
        ethn['name'] = ethn_name
        ethn['country'] = country_name
        ethn['percentage'] = elem.attrib['percentage']
        ethnic_list.append(ethn)
        
    # Populate ethnic group information in city_dict
    for elem in find_cities(country):
        city = {}
        city_name = elem.find('name').text
        city['name'] = city_name
        population_years = []
        
        for e in elem.findall('population'):
            population_years.append(int(e.attrib['year']))
            key = 'population_' + e.attrib['year']
            city[key] = e.text
        try:
            latest_year = max(population_years)
            latest_year_key = 'population_' + str(latest_year)
            city['population_latest'] = city[latest_year_key]
        except:
            city['population_latest'] = np.nan
        
        city_dict[city_name] = city
    
    # Populate country information in pop_list
    for elem in country.findall('population'):
        pop = {}
        pop['population'] = int(elem.text)
        pop['est_type'] = elem.attrib.get('measured', np.nan)
        pop['year'] = elem.attrib.get('year', np.nan)
        pop['country'] = country_name
        pop_list.append(pop)

In [367]:
#  10 countries with the lowest infant mortality rates
df = pd.DataFrame.from_dict(country_dict, orient='index')
df.infant_mortality = df.infant_mortality.astype(float)
df.sort_values(by='infant_mortality', ascending=False).head(10).infant_mortality

Western Sahara              145.82
Afghanistan                 117.23
Mali                        104.34
Somalia                     100.14
Central African Republic     92.86
Guinea-Bissau                90.92
Chad                         90.30
Niger                        86.27
Angola                       79.99
Burkina Faso                 76.80
Name: infant_mortality, dtype: float64

In [368]:
# 10 cities with the largest population
dfc = pd.DataFrame.from_dict(city_dict, orient='index')
dfc.population_latest = dfc.population_latest.astype(float)
dfc.sort_values(by='population_latest', ascending=False).head(10).population_latest

Shanghai     22315474.0
Istanbul     13710512.0
Mumbai       12442373.0
Moskva       11979529.0
Beijing      11716620.0
São Paulo    11152344.0
Tianjin      11090314.0
Guangzhou    11071424.0
Delhi        11034555.0
Shenzhen     10358381.0
Name: population_latest, dtype: float64

In [369]:
dfp = pd.DataFrame(pop_list)
dfp.head()
df_census = dfp[dfp.est_type == 'census']
df_best_pop = dfm.ix[df_census.groupby('country')['year'].idxmax(axis='year')]
df_best_pop.head()

,country,est_type,population,year
567,Afghanistan,census,13051358,1979
8,Albania,census,2800138,2011
1740,Algeria,census,34080030,2008
1389,American Samoa,census,55519,2010
60,Andorra,census,46166,1989


In [370]:
#  10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
dfe = pd.DataFrame(ethnic_list)
dfep = pd.merge(dfe, df_best_pop, on='country')
dfep.percentage = dfep.percentage.astype(float)
dfep['ethn_pop'] = ((dfep.percentage / 100.00) * dfep.population).astype(int)
summed_ethn = dfep.groupby('name').sum()
summed_ethn.ethn_pop.sort_values(ascending=False).head(10)

name
Han Chinese    1225848239
Indo-Aryan      871815583
European        473316258
Dravidian       302713744
African         288659758
Bengali         146776916
Mestizo         146528662
Russian         129730224
Japanese        127289007
Malay           120907106
Name: ethn_pop, dtype: int64

In [371]:
dfcc = pd.DataFrame.from_dict(country_dict, orient='index').reset_index().set_index('car_code')

In [372]:
# name and country of a) longest river
river_dict = {}
dfcc = pd.DataFrame.from_dict(country_dict, orient='index').reset_index().set_index('car_code')

for riv in root.iter('river'):
    river = {}
    rid = riv.attrib['id']
    river['name'] = riv.find('name').text
    river['ccodes'] = riv.attrib['country']
    
    try:
        river['length'] = riv.find('length').text
    except:
        river['length'] = np.nan
        
    
    country_names = []
    for cc in riv.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    river['country_names'] = country_names
    river_dict[rid] = river
    
dfr = pd.DataFrame.from_dict(river_dict, orient='index')
dfr.length = dfr.length.astype(float)
dfr.sort_values(by='length', ascending=False).head(10)

,country_names,length,name,ccodes
river-Amazonas,"[Colombia, Brazil, Peru]",6448.0,Amazonas,CO BR PE
river-Jangtse,[China],6380.0,Jangtse,CN
river-Hwangho,[China],4845.0,Hwangho,CN
river-Lena,[Russia],4400.0,Lena,R
river-Zaire,"[Congo, Zaire]",4374.0,Zaire,RCB ZRE
river-Mekong,"[China, Laos, Thailand, Cambodia, Vietnam]",4350.0,Mekong,CN LAO THA K VN
river-Irtysch,"[Russia, Kazakhstan, China]",4248.0,Irtysch,R KAZ CN
river-Niger,"[Mali, Niger, Nigeria, Guinea]",4184.0,Niger,RMM RN WAN RG
river-Missouri_River,[United States],4130.0,Missouri,USA
river-Jenissej,[Russia],4092.0,Jenissej,R


In [375]:
# Name and Country of b) largest lake
# TODO: Write a function to generate these dicts.

lake_dict = {}
for lk in root.iter('lake'):
    lake = {}
    lid = lk.attrib['id']
    lake['name'] = lk.find('name').text
    lake['ccodes'] = lk.attrib['country']
    
    try:
        lake['area'] = float(lk.find('area').text)
    except:
        lake['area'] = np.nan
        
    
    country_names = []
    for cc in lk.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    lake['country_names'] = country_names
    lake_dict[lid] = lake
    
dflk = pd.DataFrame.from_dict(lake_dict, orient='index')
dflk.area = dflk.area.astype(float)
dflk.sort_values(by='area', ascending=False).head(10)[['name', 'area', 'country_names']]

,name,area,country_names
lake-KaspischesMeer,Caspian Sea,386400.0,"[Russia, Azerbaijan, Kazakhstan, Iran, Turkmen..."
lake-Lake_Superior,Lake Superior,82103.0,"[Canada, United States]"
lake-Victoriasee,Lake Victoria,68870.0,"[Tanzania, Kenya, Uganda]"
lake-Lake_Huron,Lake Huron,59600.0,"[Canada, United States]"
lake-Lake_Michigan,Lake Michigan,57800.0,[United States]
lake-DeadSea,Dead Sea,41650.0,"[Israel, Jordan, West Bank]"
lake-Tanganjikasee,Lake Tanganjika,32893.0,"[Zaire, Zambia, Burundi, Tanzania]"
lake-Great_Bear_Lake,Great Bear Lake,31792.0,[Canada]
lake-Baikalsee,Ozero Baikal,31492.0,[Russia]
lake-Malawi,Lake Malawi,29600.0,"[Malawi, Mozambique, Tanzania]"


In [376]:
# Name and Country of c) airport at highest elevation
# TODO: Write a function to generate these dicts.

air_dict = {}
for ap in root.iter('airport'):
    airport = {}
    aid = ap.attrib['iatacode']
    airport['name'] = ap.find('name').text
    airport['ccodes'] = ap.attrib['country']
    
    try:
        airport['elevation'] = float(ap.find('elevation').text)
    except:
        airport['elevation'] = np.nan
        
    
    country_names = []
    for cc in ap.attrib['country'].split(' '):
        country_names.append(dfcc.loc[cc]['index'])
    
    airport['country_names'] = country_names
    air_dict[aid] = airport

dfair = pd.DataFrame.from_dict(air_dict, orient='index')
dfair.area = dfair.elevation.astype(float)
dfair.sort_values(by='elevation', ascending=False).head(10)[['name', 'elevation', 'country_names']]

,name,elevation,country_names
LPB,El Alto Intl,4063.0,[Bolivia]
LXA,Lhasa-Gonggar,4005.0,[China]
YUS,Yushu Batang,3963.0,[China]
JUL,Juliaca,3827.0,[Peru]
CUZ,Teniente Alejandro Velasco Astete Intl,3311.0,[Peru]
SRE,Juana Azurduy De Padilla,2905.0,[Bolivia]
UIO,Mariscal Sucre Intl,2813.0,[Ecuador]
AYP,Coronel Fap Alfredo Mendivil Duarte,2719.0,[Peru]
CJA,Mayor General FAP Armando Revoredo Iglesias Ai...,2677.0,[Peru]
TLC,Licenciado Adolfo Lopez Mateos Intl,2581.0,[Mexico]
